<a href="https://colab.research.google.com/github/ThrishaJawahar/Falcon-9-Launch-Prediction/blob/main/MLPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Objectives

1. Perform exploratory Data Analysis and determine Training Labels
2. create a column for the class
3. Standardize the data
4. Split into training data and test data
5. Find best Hyperparameter for SVM, Classification Trees and Logistic Regression
6. Find the method performs best using test data

TASK 1

Create a NumPy array from the column Class in data, by applying the method to_numpy() then assign it to the variable Y,make sure the output is a Pandas series (only one bracket df['name of column']).

In [3]:
import pandas as pd

URL1 = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/dataset_part_2.csv"
URL2 = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/dataset_part_3.csv"


df1 = pd.read_csv(URL1)
df2 = pd.read_csv(URL2)


Y = df1['Class'].to_numpy()


print("NumPy array Y:")
print(Y)


NumPy array Y:
[0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 1 0 0 1 1 1 1 1 0 1 1 0 1 1 0 1 1 1 0 1 1
 1 1 1 1 1 1 1 1 0 0 0 1 1 0 0 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 0 1
 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1]


TASK 2

Standardize the data in X then reassign it to the variable X using the transform provided below.

In [4]:
from sklearn.preprocessing import StandardScaler

X = df1[['FlightNumber', 'PayloadMass', 'Orbit', 'LaunchSite', 'Flights', 'GridFins', 'Reused', 'Legs', 'LandingPad', 'Block', 'ReusedCount', 'Serial']]

X = pd.get_dummies(X, columns=['Orbit', 'LaunchSite', 'LandingPad', 'Serial'])

scaler = StandardScaler()
X = scaler.fit_transform(X)

X = pd.DataFrame(X, columns=scaler.feature_names_in_)
print("Standardized X:")
print(X.head())

Standardized X:
   FlightNumber   PayloadMass   Flights  ...  Serial_B1059  Serial_B1060  Serial_B1062
0     -1.712912 -1.948145e-16 -0.653913  ...     -0.215666     -0.185695        -0.106
1     -1.674419 -1.195232e+00 -0.653913  ...     -0.215666     -0.185695        -0.106
2     -1.635927 -1.162673e+00 -0.653913  ...     -0.215666     -0.185695        -0.106
3     -1.597434 -1.200587e+00 -0.653913  ...     -0.215666     -0.185695        -0.106
4     -1.558942 -6.286706e-01 -0.653913  ...     -0.215666     -0.185695        -0.106

[5 rows x 80 columns]


TASK 3

Use the function train_test_split to split the data X and Y into training and test data. Set the parameter test_size to 0.2 and random_state to 2. The training data and test data should be assigned to the following labels.

X_train, X_test, Y_train, Y_test

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("Y_train shape:", Y_train.shape)
print("Y_test shape:", Y_test.shape)

X_train shape: (72, 80)
X_test shape: (18, 80)
Y_train shape: (72,)
Y_test shape: (18,)


TASK 4

Create a logistic regression object then create a GridSearchCV object logreg_cv with cv = 10. Fit the object to find the best parameters from the dictionary parameters.



In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

logreg = LogisticRegression(max_iter=1000)

parameters = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l2'],
    'solver': ['lbfgs']
}


logreg_cv = GridSearchCV(logreg, param_grid=parameters, cv=10, scoring='accuracy')

logreg_cv.fit(X_train, Y_train)

print("Best parameters found:", logreg_cv.best_params_)
print("Best cross-validation accuracy:", logreg_cv.best_score_)

Best parameters found: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
Best cross-validation accuracy: 0.8214285714285714


TASK 5

Calculate the accuracy on the test data using the method score:

In [7]:
test_accuracy = logreg_cv.score(X_test, Y_test)

print(f"Accuracy on test data: {test_accuracy:.4f}")


Accuracy on test data: 0.8333


TASK 6

Create a support vector machine object then create a GridSearchCV object svm_cv with cv = 10. Fit the object to find the best parameters from the dictionary parameters.

In [8]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

parameters = {'C': [0.1, 1, 10, 100], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}

svm = SVC()

svm_cv = GridSearchCV(estimator=svm, param_grid=parameters, cv=10, scoring='accuracy')

svm_cv.fit(X_train, Y_train)

print(f"Best Parameters: {svm_cv.best_params_}")
print(f"Best Score: {svm_cv.best_score_:.4f}")

Best Parameters: {'C': 1, 'kernel': 'sigmoid'}
Best Score: 0.8482


TASK 7

Calculate the accuracy on the test data using the method score:

In [9]:
test_accuracy = svm_cv.score(X_test, Y_test)

print(f"Accuracy on test data: {test_accuracy:.4f}")



Accuracy on test data: 0.8333


TASK 8

Create a decision tree classifier object then create a GridSearchCV object tree_cv with cv = 10. Fit the object to find the best parameters from the dictionary parameters.



In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

tree = DecisionTreeClassifier()

parameters = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [2, 4, 6, 8, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

tree_cv = GridSearchCV(tree, parameters, cv=10, scoring='accuracy')
tree_cv.fit(X_train, Y_train)
print("Best parameters found:", tree_cv.best_params_)
print("Best accuracy score from GridSearchCV:", tree_cv.best_score_)


Best parameters found: {'criterion': 'entropy', 'max_depth': 8, 'min_samples_leaf': 4, 'min_samples_split': 5}
Best accuracy score from GridSearchCV: 0.8714285714285713


TASK 9

Calculate the accuracy of tree_cv on the test data using the method score:


In [11]:
accuracy = tree_cv.score(X_test, Y_test)
print("Accuracy of the Decision Tree on test data:", accuracy)

Accuracy of the Decision Tree on test data: 0.7777777777777778


TASK 10

Create a k nearest neighbors object then create a GridSearchCV object knn_cv with cv = 10. Fit the object to find the best parameters from the dictionary parameters.



In [12]:
from sklearn.neighbors import KNeighborsClassifier

# Define the parameter grid for KNN
knn_parameters = {'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
              'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
              'p': [1,2]}


# Create the KNN classifier object
knn = KNeighborsClassifier()

# Create the GridSearchCV object
from sklearn.model_selection import GridSearchCV
knn_cv = GridSearchCV(knn, knn_parameters, cv=10)

# Fit the GridSearchCV object
knn_cv.fit(X_train, Y_train)

# Display the best parameters
print("Best parameters for KNN:", knn_cv.best_params_)


Best parameters for KNN: {'algorithm': 'auto', 'n_neighbors': 6, 'p': 1}


TASK 11

Calculate the accuracy of knn_cv on the test data using the method score:



In [13]:
knn_accuracy = knn_cv.score(X_test, Y_test)
print(f"Accuracy of KNN on the test data: {knn_accuracy * 100:.2f}%")

Accuracy of KNN on the test data: 83.33%


TASK 12

Find the method performs best:

In [14]:
logreg_accuracy = logreg_cv.score(X_test, Y_test)
svm_accuracy = svm_cv.score(X_test, Y_test)
tree_accuracy = tree_cv.score(X_test, Y_test)
knn_accuracy = knn_cv.score(X_test, Y_test)
model_accuracies = {
    "Logistic Regression": logreg_accuracy,
    "Support Vector Machine": svm_accuracy,
    "Decision Tree": tree_accuracy,
    "K-Nearest Neighbors": knn_accuracy
}
best_model = max(model_accuracies, key=model_accuracies.get)
best_accuracy = model_accuracies[best_model]
print(f"The best performing model is: {best_model} with an accuracy of {best_accuracy * 100:.2f}%")


The best performing model is: Logistic Regression with an accuracy of 83.33%


In [15]:
num_records_test = X_test.shape[0]
print(f"There are {num_records_test} records in the test sample.")

There are 18 records in the test sample.
